In [1]:
from platypus import NSGAII, Problem, Real
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import pandas as pd
import random

In [2]:
# Load the autoreload extension
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
from Mult_Obj_Helathy import Main_Model as Mdl_H 
from Mult_Obj_Tumor import Main_Model as Mdl_T

# Mixture of CAR NK and WT
(1) check for 50 and 50 

In [4]:
Shell_H = Mdl_H()
Shell_T = Mdl_T()

In [5]:
Tumor_No = 10000
for i in [10.0,5.0,2.5,1.25,0.625,0.3125]:
    x0 = i
    NK_No = x0*Tumor_No
    x = [0.6,0.6]
    param_100 = np.array([0.01, 1.3611247151766386, 2.646954531914279, 1.119928427439465, 0.8030447162325958])   
    frac = param_100[-1]
    MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 40), (0, 45), (0, 200.0)]]
    Shell_H.Healthy(x[0], frac, x[1], NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Monocyte',limts=MFI_limt)
    MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 30), (0, 70), (0, 20.0)]]
    Shell_T.Tumor(x[0], frac, x[1], NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Kasumi1',limts=MFI_limt)
    print('done')

H 68.53362691297595
T 55.17716270725437
done
H 57.51010394239853
T 43.939986127538454
done
H 45.47040999011317
T 32.56247616723873
done
H 33.113697526729155
T 21.502945682262485
done
H 21.526976175280765
T 12.717272134582448
done
H 12.608158197040432
T 6.973985832336293
done


In [6]:
# Define your multi-objective problem here
class MyProblem(Problem):
    def __init__(self,ET_ratio):
        super(MyProblem, self).__init__(1, 2) # #variables, #objectives)
        self.types[:] = [Real(0.001, 0.8)]
        self.Shell_H = Mdl_H()
        self.Shell_T = Mdl_T()
        self.ET_ratio = ET_ratio
        self.directions[:] = [Problem.MINIMIZE, Problem.MAXIMIZE]

    def evaluate(self, solution):
        x = np.array(solution.variables)
        print('x0',x)
        Tumor_No = 10000
        NK_No = self.ET_ratio*Tumor_No
        param_100 = np.array([0.01, 1.3611247151766386, 2.646954531914279, 1.119928427439465, 0.8030447162325958]) 
        #frac = param_100[-1]
        alpha1 = 0.6
        alpha4 = 0.6
        MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 40), (0, 45), (0, 200.0)]]
        f1 = self.Shell_H.Healthy(alpha1, x[0], alpha4, NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Monocyte',limts=MFI_limt) # Objective 1
        MFI_limt = [[(0, 100), (0, 300), (0.0, 50.0)], [(0, 30), (0, 70), (0, 20.0)]]
        f2 = self.Shell_T.Tumor(alpha1, x[0], alpha4, NK_No = NK_No, Tc_No = Tumor_No, param=param_100[:-1], NK_cell='MIX', Tumor_cell='Kasumi1',limts=MFI_limt) # Objective 2
        solution.objectives[:] = [f1, f2]

In [7]:
seed_value = 42
np.random.seed(seed_value)
random.seed(seed_value)
for ET_ratio in [1,5,10]:
        problem = MyProblem(ET_ratio)
        algorithm = NSGAII(problem)
        algorithm.run(2)
        Healthy_Lysis = [s.objectives[0] for s in algorithm.result]
        Tumor_Lysis = [s.objectives[1] for s in algorithm.result]
        w =  [s.variables[0] for s in algorithm.result]
        filename = f'w_p001_1_Lysis_Kasumi_Healthy_CAR_NK_a4_ET_{ET_ratio}.xlsx'
        df = pd.DataFrame(columns=['w','Tumor_lysis', 'Healthy_Lysis'])
        df['w'] = w
        df['Tumor_lysis'] = Tumor_Lysis
        df['Healthy_Lysis'] = Healthy_Lysis
        df.to_excel(filename)
        print('Done for', ET_ratio)

x0 [0.51190201]
H 22.46067754537021
T 15.038285739526868
x0 [0.02098359]
H 5.341540613343387
T 8.515700970439099
x0 [0.22074843]
H 13.502300589073768
T 11.325897296262754
x0 [0.17934538]
H 11.930276688069885
T 10.75736904297142
x0 [0.5894405]
H 24.432459384746796
T 15.959987137455755
x0 [0.54168289]
H 23.252744407664714
T 15.395868914693168
x0 [0.71385147]
H 27.302351779733236
T 17.37455204449223
x0 [0.07046413]
H 7.518703457617326
T 9.229390294832573
x0 [0.33811553]
H 17.374431998262065
T 12.870061696979063
x0 [0.02480798]
H 5.4975022730200385
T 8.571412054420602
x0 [0.17569174]
H 11.825968598534796
T 10.714631836586886
x0 [0.40477888]
H 19.46320965647216
T 13.720179183928426
x0 [0.02220224]
H 5.402802594741807
T 8.536243181006009
x0 [0.15987128]
H 11.210636831817899
T 10.491945810989268
x0 [0.52025767]
H 22.660799832969925
T 15.129409419085604
x0 [0.43640824]
H 20.354487526580634
T 14.110927520025363
x0 [0.17713206]
H 11.904321400621553
T 10.733416301503473
x0 [0.47182328]
H 21.32945